# Worksheet 08

Name: Jin Young Bang

UID: U26738680

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.391445094733568, 3.9968437611534178, 4.273857480802641, 2.7630243884657975, 4.5235295253982555, 5.781617494797077, 4.512723447615677, 5.5532393628823655, 4.590678154158948, 4.883261377400749]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.399112760727791, 9.528349924633211, 9.916334338245132, 7.046710896593468, 7.211471635779335, 6.948865124393702, 6.235915246407661, 5.3673642458627056, 8.72021184178577, 6.95548312227602]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.883261377400749, 4.590678154158948, 5.5532393628823655, 6.95548312227602, 4.512723447615677, 8.72021184178577, 5.781617494797077, 4.5235295253982555, 2.7630243884657975, 5.3673642458627056]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- The number of components (K) = 2
- Weights (1/2, 1/2)
- Component Means (5, 8)
- Component Variances (1, 1)

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1) / len(s1) , sum(s2) / len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.883261377400749, 4.590678154158948, 5.5532393628823655, 4.512723447615677, 5.781617494797077, 4.5235295253982555, 2.7630243884657975, 5.3673642458627056]
[6.95548312227602, 8.72021184178577]
P(S_1) = 0.8,  P(S_2) = 0.2
mean_1 = 4.746929749572697,  mean_2 = 7.837847482030895
var_1 = 0.7736618118126992,  var_2 = 10.332339292190161


/opt/homebrew/Cellar/jupyterlab/4.0.6/libexec/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


- s1: mean=5, var=1
- s2: mean=8, var=1

The estimates for the mean are close and the variances are far from the true values

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [7]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.883261377400749
probability of observing that point if it came from cluster 0 =  0.5077103634612187
probability of observing that point if it came from cluster 1 =  0.03706424821922935
point =  4.590678154158948
probability of observing that point if it came from cluster 0 =  0.5052445229628681
probability of observing that point if it came from cluster 1 =  0.03675059637730843
point =  5.5532393628823655
probability of observing that point if it came from cluster 0 =  0.29956988870022344
probability of observing that point if it came from cluster 1 =  0.03767861492323982
point =  6.95548312227602
probability of observing that point if it came from cluster 0 =  0.008765742701520822
probability of observing that point if it came from cluster 1 =  0.03847049546982546
point =  4.512723447615677
probability of observing that point if it came from cluster 0 =  0.4925599373443679
probability of observing that point if it came from cluster 1 =  0.036662516762905085
point =  8.72021

It seems that generally, the data points are confidently assigned to a cluster.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [8]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), ... ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.8,  P(S_2) = 0.2
mean_1 = 4.945501856495912,  mean_2 = 7.042593298569119
var_1 = 0.8641496081530786,  var_2 = 4.538492513539249


Apart from mean of cluster 1, it seems that the values are pretty different compared to the values from KMeans. Additionally, the variance seems to be much more bigger as well for cluster 2.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [9]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.883261377400749
probability of observing that point if it came from cluster 0 =  0.46046290762994846
probability of observing that point if it came from cluster 1 =  0.07849521726719054
point =  4.590678154158948
probability of observing that point if it came from cluster 0 =  0.4243370603117311
probability of observing that point if it came from cluster 1 =  0.07596611990581281
point =  5.5532393628823655
probability of observing that point if it came from cluster 0 =  0.36051229471000523
probability of observing that point if it came from cluster 1 =  0.08329405517007388
point =  6.95548312227602
probability of observing that point if it came from cluster 0 =  0.03086956688497921
probability of observing that point if it came from cluster 1 =  0.08788574657134529
point =  4.512723447615677
probability of observing that point if it came from cluster 0 =  0.40724650833194304
probability of observing that point if it came from cluster 1 =  0.075253351268652
point =  8.7202118

Although the probability of the confidence lowered a bit, I think this is showing the adaptability of potential overlapping clusters from the GMM model.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [12]:
print(data)
hard_assignments = [[], []]

for d, p in zip(data, prob_s0_x):
    if random.random() <= p: hard_assignments[0].append(d)
    else: hard_assignments[1].append(d)

print()
print(f"Cluster 0 Points: {hard_assignments[0]}")
print(f"Cluster 1 Points: {hard_assignments[1]}")

[4.883261377400749, 4.590678154158948, 5.5532393628823655, 6.95548312227602, 4.512723447615677, 8.72021184178577, 5.781617494797077, 4.5235295253982555, 2.7630243884657975, 5.3673642458627056]

Cluster 0 Points: [4.883261377400749, 4.590678154158948, 5.5532393628823655, 4.512723447615677, 5.781617494797077, 4.5235295253982555, 2.7630243884657975, 5.3673642458627056]
Cluster 1 Points: [6.95548312227602, 8.72021184178577]
